In [1]:
cd ../..

/home/users/cdonoso/astromer


/home/users/cdonoso/miniconda3/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd 
import toml 
import os

import matplotlib.pyplot as plt 
from src.data.record import DataPipeline

from src.models.astromer_1 import get_ASTROMER

In [5]:
model = get_ASTROMER(num_layers=6,
                 num_heads=5,
                 head_dim=64,
                 mixer_size=512,
                 dropout=0.1,
                 pe_base=1000,
                 pe_dim=128,
                 pe_c=1,
                 window_size=200,
                 batch_size=None,
                 m_alpha=-1000000000,
                 mask_format='K',
                 use_leak=True,
                 loss_format='rmse',
                 correct_loss=False,
                 trainable_mask=True,
                 temperature=0.)

[INFO] Temperature: 0.00
[INFO] Mask format: K
[INFO] Adding trainable MSK token


In [6]:
model.summary()

Model: "ASTROMER-1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 200, 1)]             0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 200, 1)]             0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 200, 1)]             0         []                            
                                                                                                  
 encoder (Encoder)           (None, 200, 320)             5431168   ['input_4[0][0]',             
                                                                     'input_5[0][0]',    

In [3]:
data = './data/raw_parquet/alcock/'
target_folder = './data/records/alcock'


metapath = os.path.join(data, 'metadata.parquet')
lcspath  = os.path.join(data, 'light_curves')


basename = os.path.basename(os.path.normpath(target_folder))

In [5]:
os.path.join(target_folder, )

'./data/raw_parquet/alcock/metadata.parquet'

In [13]:
import shutil
for spc in [20, 100, 500, '']:  
    if spc == '':
        spc = 'all'
    for foldn in range(5):
        source = os.path.join('./data/shared/records/alcock/fold_0/alcock_100/objects.csv')
        pathcur = os.path.join(target_folder, str(spc), 'fold_{}'.format(foldn), 'objects.csv')
        print(pathcur)
        shutil.copyfile(source, pathcur)

./data/records/alcock/20/fold_0/objects.csv
./data/records/alcock/20/fold_1/objects.csv
./data/records/alcock/20/fold_2/objects.csv
./data/records/alcock/20/fold_3/objects.csv
./data/records/alcock/20/fold_4/objects.csv
./data/records/alcock/100/fold_0/objects.csv
./data/records/alcock/100/fold_1/objects.csv
./data/records/alcock/100/fold_2/objects.csv
./data/records/alcock/100/fold_3/objects.csv
./data/records/alcock/100/fold_4/objects.csv
./data/records/alcock/500/fold_0/objects.csv
./data/records/alcock/500/fold_1/objects.csv
./data/records/alcock/500/fold_2/objects.csv
./data/records/alcock/500/fold_3/objects.csv
./data/records/alcock/500/fold_4/objects.csv
./data/records/alcock/all/fold_0/objects.csv
./data/records/alcock/all/fold_1/objects.csv
./data/records/alcock/all/fold_2/objects.csv
./data/records/alcock/all/fold_3/objects.csv
./data/records/alcock/all/fold_4/objects.csv


In [5]:
with open('./data/config.toml', 'r') as file:
    config = toml.load(file)

valfrac = 0.2
n_folds = 5

for spc in [20, 100, 500, '']:    
    metadata = pd.read_parquet(metapath)
    
    for nfold in range(n_folds):
        test_metadata = metadata.groupby('Label').sample(n=100)
        rest_metadata = metadata[~metadata['newID'].isin(test_metadata['newID'])]
    
        # Train Val and Test splits
        if spc != '':
            nval = int(valfrac*spc)
            valid_meta = rest_metadata.sample(n=nval)
            train_meta = rest_metadata[~rest_metadata['newID'].isin(valid_meta['newID'])].groupby('Label').sample(n=spc,
                                                                                                                  replace=True)
        else:
            nval = int(valfrac*rest_metadata.shape[0])
            valid_meta = rest_metadata.sample(n=nval)
            train_meta = rest_metadata[~rest_metadata['newID'].isin(valid_meta['newID'])]
            
        print(train_meta.shape, valid_meta.shape, test_metadata.shape)

        train_meta    = train_meta.copy()
        valid_meta    = valid_meta.copy()
        test_metadata = test_metadata.copy()
        
        train_meta.loc[:, 'subset_{}'.format(nfold)]    = ['train']*train_meta.shape[0]
        valid_meta.loc[:, 'subset_{}'.format(nfold)]    = ['validation']*valid_meta.shape[0]
        test_metadata.loc[:, 'subset_{}'.format(nfold)] = ['test']*test_metadata.shape[0]

        curr_meta = pd.concat([train_meta, valid_meta, test_metadata], axis=0)
        cols_to_use = curr_meta.columns.difference(metadata.columns)
        metadata = pd.merge(metadata, curr_meta[cols_to_use], left_index=True, right_index=True, how='outer')

    # ============== Pipeline ==============
    # ======================================
    if spc == '': spc = 'all'
    target_path = os.path.join(target_folder, str(spc))
    os.makedirs(target_path, exist_ok=True)

    metadata.to_parquet(os.path.join(target_path, 'metadata.parquet'), index=False)
    
    config['target']['path']              = target_path
    config['context_features']['path']    = os.path.join(target_folder, str(spc), 'metadata.parquet')
    config['sequential_features']['path'] = lcspath

    configPath = os.path.join(target_path, 'config.toml')
    with open(configPath, 'w') as file:
        toml.dump(config, file)

    pipeline = DataPipeline(metadata=metadata,
                            config_path=configPath)
    
    var = pipeline.run(n_jobs=4,
                       elements_per_shard=200000)

    break

(140, 9) (4, 9) (700, 9)
(140, 10) (4, 10) (700, 10)
(140, 11) (4, 11) (700, 11)
(140, 12) (4, 12) (700, 12)
(140, 13) (4, 13) (700, 13)


2024-11-05 17:28:31,015 - INFO - Starting DataPipeline operations


[WARN] Inconsistent data types in context. Overwritting config...
[INFO] 21450 samples loaded
[INFO] Reading parquet
[INFO] Light curves loaded


Processing fold 1/5 - test:   0%|                                                                                                                  | 0/3 [00:00<?, ?it/s]2024-11-05 17:28:56.367242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-05 17:28:56.368150: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-05 17:28:56.401521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative 

In [ ]:
metadata

In [8]:
fold_groups = [x for x in metadata.columns if 'subset' in x]

In [9]:
fold_groups

['subset_0', 'subset_1', 'subset_2', 'subset_3', 'subset_4']

In [10]:
metadata[fold_groups[0]].dropna().unique()

array(['test', 'train', 'validation'], dtype=object)

In [11]:
partial = metadata[metadata['subset_1'] == 'train']